# Predicción Bancarrota de Empresas
## Análisis de 6.800 empresas

El siguiente conjunto de datos son **6.800 entradas de empresas de Taiwan con sus características distribuidas en 95 columnas**. De ellas 220 entraron en bancarrota de modo que el **objetivo** de este proyecto es **predecir si una empresa se ira a la bancarrota o no**, siendo las columnas sus variables independientes de las que dependa la predicción.

link data: https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction

Para realizar la predicción más precisa he utilizado distintos tipos de algoritmos para así tener el mejor modelo entrenado que me interese en función de la mejor predicción. Por ejemplo: **Xgboost, Red Neuronal Artificial con distintos optimizadores, Teorema de Bayes, Regresión Logística, Máquina Soporte Vectorial, etc.**

### Librerías

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

### Clase Algoritmos
Esta clase se encargará de calcular la precisión de los algoritmos siempre y cuando inicialice correctamente el constructor con sus parametros correctos. Después solo hay que llamar a la funcion **precision()**. También almacenará los datos de precisión en un dataframe **df** para una consulta posterior en el apartado de la conclusión.

In [2]:
datos = {'Nombre_Algoritmo':[], 'Precision':[]}  
df = pd.DataFrame(data=datos) 
class Algoritmos:
    
    def __init__(self,nombre,cm, y_test):
        self.nombre = nombre
        self.cm = cm
        self.y_test = y_test
        
    def __str__(self):
        return (self.nombre)
        
    def precision(self):
        global df
        assert(self.cm.shape==(2,2))
        assert(type(self.y_test) == pd.DataFrame)
        assert(type(self.cm[0][0]) == np.int64)
        precision_02 = ((self.cm[0][0] + self.cm[1][1])/len(self.y_test)) * 100
        assert(type(precision_02) == np.float64)
        df = df.append({'Nombre_Algoritmo':self.nombre, 'Precision':precision_02}, ignore_index=True)
        return("%.2f" %precision_02)

###  Carga y Visualización de datos

In [3]:
# PATH
path = "C:/Users/USUARIO/Desktop/BancarrotaEmpresas/"
data = pd.read_csv(path + "data.csv")

In [4]:
data # 6819 entrados de datos con 96 columnas cada una de ellas

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


In [5]:
p = data['Bankrupt?'] == 1 # Empresas que han sufrido bancarrota
filtro = data[p]
len(filtro.index) # 220 empresas de las 6819 empresas

220

In [6]:
data.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,...,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.0,6819.000000
mean,0.032263,0.505180,0.558625,0.553589,0.607948,0.607929,0.998755,0.797190,0.809084,0.303623,...,0.807760,1.862942e+07,0.623915,0.607946,0.840402,0.280365,0.027541,0.565358,1.0,0.047578
std,0.176710,0.060686,0.065620,0.061595,0.016934,0.016916,0.013010,0.012869,0.013601,0.011163,...,0.040332,3.764501e+08,0.012290,0.016934,0.014523,0.014463,0.015668,0.013214,0.0,0.050014
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
25%,0.000000,0.476527,0.535543,0.527277,0.600445,0.600434,0.998969,0.797386,0.809312,0.303466,...,0.796750,9.036205e-04,0.623636,0.600443,0.840115,0.276944,0.026791,0.565158,1.0,0.024477
50%,0.000000,0.502706,0.559802,0.552278,0.605997,0.605976,0.999022,0.797464,0.809375,0.303525,...,0.810619,2.085213e-03,0.623879,0.605998,0.841179,0.278778,0.026808,0.565252,1.0,0.033798
75%,0.000000,0.535563,0.589157,0.584105,0.613914,0.613842,0.999095,0.797579,0.809469,0.303585,...,0.826455,5.269777e-03,0.624168,0.613913,0.842357,0.281449,0.026913,0.565725,1.0,0.052838
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,9.820000e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000


### Preprocesado de datos

Primero compruebo si hay algun valor nulo y duplicado en mi conjunto de datos. Luego observo el tipo de datos por si existiera alguna columna tipo object para transformarla. En este caso no es necesario ya que todas son entradas de datos tipo float64 e int64. 

In [7]:
# Preprocesado
data.duplicated().sum() # ninguno

0

In [8]:
data.isnull().sum() # ninguno

Bankrupt?                                                   0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and depreciation after tax          0
 Operating Gross Margin                                     0
                                                           ..
 Liability to Equity                                        0
 Degree of Financial Leverage (DFL)                         0
 Interest Coverage Ratio (Interest expense to EBIT)         0
 Net Income Flag                                            0
 Equity to Liability                                        0
Length: 96, dtype: int64

In [9]:
data.dtypes

Bankrupt?                                                     int64
 ROA(C) before interest and depreciation before interest    float64
 ROA(A) before interest and % after tax                     float64
 ROA(B) before interest and depreciation after tax          float64
 Operating Gross Margin                                     float64
                                                             ...   
 Liability to Equity                                        float64
 Degree of Financial Leverage (DFL)                         float64
 Interest Coverage Ratio (Interest expense to EBIT)         float64
 Net Income Flag                                              int64
 Equity to Liability                                        float64
Length: 96, dtype: object

### Transformacíon de datos

Almaceno las columnas que considero variables independientes y dependiente, por ejemplo en este supuesto todas las columnas son importantes en el problema de modo que no me deshago de ninguna. El siguiente paso es considerar quien es la variable dependiente y las variables independientes:

- El objetivo de este supesto es predecir la columna **'Bankrupt?'** para pronosticar si una empresa entrará en bancarrota o no . De modo que la columna **'Bankrupt?' será la variable dependiente**.

- Como he dicho antes, todas las columnas tienen relevancia en este problema entonces **las variables independientes seran todas las columnas menos 'Bankrupt'**

In [10]:
X = data.iloc[:, 1:-1]
Y = data.iloc[:, 0:1]

In [11]:
X

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag
0,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,...,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1
1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,...,0,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1
2,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,0.780284,...,0,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1
3,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,0.781241,...,0,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1
4,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,0.781550,...,0,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,0.781588,...,0,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1
6815,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,0.781586,...,0,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1
6816,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,0.781546,...,0,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1
6817,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,0.781663,...,0,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1


In [12]:
Y

,Bankrupt?
0,1
1,1
2,1
3,1
4,1
...,...
6814,0
6815,0
6816,0
6817,0


### TRAIN Y TEST
Después de visualizar los datos, preprocesarlos, considerar que columnas son relevantes para el caso de estudio y dividirlos en dos matrices ahora ya se pasaría a la fase de división de datos, es decir, considerar cuanto porcentaje de datos del dataset sería para entrenar el modelo, otro porcentaje para testearlo y verificar su precisión. Normalmente:

- **TRAIN**, tiene el **80% de datos del dataset original** ya que cuanto más datos más comprobaciones realizará y más fiable será.
- **TEST**, tiene el **20% de datos del dataset original** y con él se mide la precisión.

Como sería un proceso muy laborioso hacerlo manualmente con está gran cantidad de datos hay una función en Python del modulo **sklearn** llamada **train_test_split** que nos facilita esta división de datos.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=(0))

# Escalado de variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# test 1364 de empresas
# train 5455 de empresas

In [14]:
p = y_test['Bankrupt?'] == 1 # Empresas que han sufrido bancarrota del test
filtro = y_test[p]
len(filtro.index) # 46 empresas en bancarrota de las 1364 empresas

46

### Análisis discriminante lineal
El análisis discriminante lineal (Linear Discriminant Analysis o LDA) es un método alternativo más adecuado a la regresión logística cuando la variable cualitativa tiene más de dos niveles (K ≥ 2). Supone también un modelo más estable cuando el tamaño muestral n es pequeño y la distribución de los predictores es aproximadamente normal en cada una de sus clases. El propósito del LDA es encontrar la combinación lineal de las variables originales que permita la mejor separación entre grupos de un set de datos.

In [15]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

y_pred = lda.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #  96.77%  de precision
cm

array([[1304,   14],
       [  30,   16]], dtype=int64)

In [16]:
cm[1][1]

16

In [17]:
cl = Algoritmos("Análisis discriminante lineal", cm, y_test)
cl.precision()

'96.77'

### AdaBoost
Un clasificador AdaBoost es un metaestimador que comienza ajustando un clasificador en el conjunto de datos original y luego ajusta copias adicionales del clasificador en el mismo conjunto de datos, pero donde los pesos de las instancias clasificadas incorrectamente se ajustan de modo que los clasificadores posteriores se centren más en casos difícile

In [18]:
ad = AdaBoostClassifier(random_state=0)
ad.fit(X_train, y_train)

y_pred = ad.predict(X_test)

cm = confusion_matrix(y_test, y_pred) #  96.41%  de precision
cm

array([[1302,   16],
       [  33,   13]], dtype=int64)

In [19]:
cm[1][1]

13

In [20]:
ada = Algoritmos("Adaboost", cm, y_test)
ada.precision()

'96.41'

### Algoritmo Regresión Logística
La regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras.

In [21]:
LogisticR = LogisticRegression(random_state=0) # random state = 0
LogisticR.fit(X_train, y_train) #Ajustamos el modelo

# predicción de los resultados con el conjunto de testing
y_pred = LogisticR.predict(X_test)

# Matriz de confusion
cm = confusion_matrix(y_test, y_pred) 
cm

array([[1311,    7],
       [  39,    7]], dtype=int64)

In [22]:
cm[1][1]

7

In [23]:
Reg = Algoritmos("Regresión Logística", cm, y_test)
Reg.precision() 

'96.63'

### Algoritmo KNN (K Vecinos más cercanos)

Este algoritmo es un clasificador de aprendizaje supervisado no paramétrico, que utiliza la proximidad para hacer clasificaciones o predicciones sobre la agrupación de un punto de datos individual. Si bien se puede usar para problemas de regresión o clasificación, generalmente se usa como un algoritmo de clasificación, partiendo de la suposición de que se pueden encontrar puntos similares cerca uno del otro.

In [24]:
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # Objeto clasifier
KNN.fit(X_train, y_train)

y_pred = KNN.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #  99.95% de precision
cm

array([[1310,    8],
       [  36,   10]], dtype=int64)

In [25]:
cm[1][1]

10

In [26]:
KNN = Algoritmos("KNV", cm, y_test) 
KNN.precision()

'96.77'

### Algoritmo Teorema de Bayes
El teorema de Bayes es utilizado para calcular la probabilidad de un suceso, teniendo información de antemano sobre ese suceso. Podemos calcular la probabilidad de un suceso A, sabiendo además que ese A cumple cierta característica que condiciona su probabilidad. El teorema de Bayes entiende la probabilidad de forma inversa al teorema de la probabilidad total. El teorema de la probabilidad total hace inferencia sobre un suceso B, a partir de los resultados de los sucesos A. Por su parte, Bayes calcula la probabilidad de A condicionado a B.

In [27]:
BAYES = GaussianNB()
BAYES.fit(X_train, y_train)

y_pred = BAYES.predict(X_test)
cm = confusion_matrix(y_test, y_pred) #97.82 % de precision
cm

array([[324, 994],
       [  5,  41]], dtype=int64)

In [28]:
cm[1][1]

41

In [29]:
TeorBayes = Algoritmos("Teorema de Bayes", cm, y_test)
TeorBayes.precision()

'26.76'

### Árbol de Clasificación
Un árbol de clasificación es un tipo de árbol de decisiones. Utiliza la medida de impurezas de Gini para clasificar los registros en las categorías del campo objetivo. Las predicciones se basan en combinaciones de valores en los campos de entrada.
Un árbol de clasificación calcula la categoría de destino pronosticada para cada nodo en el árbol.

In [30]:
DecisionTree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DecisionTree.fit(X_train, y_train)

y_pred = DecisionTree.predict(X_test)
cm = confusion_matrix(y_test, y_pred)  # 99,92% de precision
cm

array([[1288,   30],
       [  31,   15]], dtype=int64)

In [31]:
cm[1][1]

15

In [32]:
Arbol = Algoritmos("Árboles de Clasificación", cm, y_test)
Arbol.precision()

'95.53'

In [33]:
# Aleatorio Entropia
rand = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rand.fit(X_train, y_train)

y_pred = rand.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 99.95% precision
cm

array([[1309,    9],
       [  41,    5]], dtype=int64)

In [34]:
cm[1][1]

5

In [35]:
ArbolRandomEntropia = Algoritmos("Árboles Aleatorios Entropia", cm, y_test)
ArbolRandomEntropia.precision()

'96.33'

In [36]:
# Aleatorio Gini
rand02 = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
rand02.fit(X_train, y_train)

y_pred = rand02.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # 99.94%% precision
cm

array([[1312,    6],
       [  35,   11]], dtype=int64)

In [37]:
cm[1][1]

11

In [38]:
ArbolRandomGini = Algoritmos("Árboles Aleatorios Gini", cm, y_test)
ArbolRandomGini.precision()

'96.99'

### Algoritmo de XGBOOST
XGBoost es una biblioteca de aumento de gradiente distribuida optimizada diseñada para ser altamente eficiente , flexible y portátil . Implementa algoritmos de aprendizaje automático bajo el marco Gradient Boosting . XGBoost proporciona un impulso de árbol paralelo (también conocido como GBDT, GBM) que resuelve muchos problemas de ciencia de datos de una manera rápida y precisa

In [39]:
XGBOOST = XGBClassifier()
XGBOOST.fit(X_train, y_train)

y_pred = XGBOOST.predict(X_test)
cm = confusion_matrix(y_test, y_pred) # % precision
cm

array([[1303,   15],
       [  36,   10]], dtype=int64)

In [40]:
cm[1][1]

10

In [41]:
xgb = Algoritmos("XGBOOST", cm, y_test)
xgb.precision()

'96.26'

### Algoritmo Red Neuronal Artificial
Una red neuronal es un modelo simplificado que emula el modo en que el cerebro humano procesa la información: Funciona simultaneando un número elevado de unidades de procesamiento interconectadas que parecen versiones abstractas de neuronas.
La red aprende examinando los registros individuales, generando una predicción para cada registro y realizando ajustes a las ponderaciones cuando realiza una predicción incorrecta. Este proceso se repite muchas veces y la red sigue mejorando sus predicciones hasta haber alcanzado uno o varios criterios de parada.

In [42]:
# red neuronal artificial con keras del modelo secuencial
RNA = tf.keras.models.Sequential()

# 1Âº Capa de inputs
RNA.add(tf.keras.layers.Dense(units=20, activation='relu'))
# 2Âº Capa oculta de 6 nodos
RNA.add(tf.keras.layers.Dense(units=10, activation='relu'))
# 3º Capa
RNA.add(tf.keras.layers.Dense(units=5, activation='relu'))
# Ultima capa de salidad, con funcion sigmoide
RNA.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


diccionario = {0:'Adadelta',1:'Adagrad',2:'Adam',3:'Adamax',4:'Ftrl',5:'Nadam',6:'RMSprop',7:'SGD'}
for optimi in diccionario.values():
    RNA.compile(optimizer = optimi, loss = 'binary_crossentropy', metrics = ['accuracy'])
    RNA.fit(X_train, y_train, batch_size = 64, epochs = 100)
    
    y_pred = RNA.predict(X_test)
    y_pred_02 = (y_pred > 0.5)
    y_pred_02 = np.where(y_pred_02=='True',1,y_pred_02)
    cm = confusion_matrix(y_test, y_pred_02)    
    redneuro = Algoritmos("Red Neuronal Artificial Optimizador: " + str(optimi), cm, y_test)
    redneuro.precision()

Epoch 1/100
86/86 [==============================] - 2s 3ms/step - loss: 0.7654 - accuracy: 0.3182
Epoch 2/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7638 - accuracy: 0.3232
Epoch 3/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7622 - accuracy: 0.3274
Epoch 4/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7607 - accuracy: 0.3335
Epoch 5/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7590 - accuracy: 0.3384
Epoch 6/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7574 - accuracy: 0.3412
Epoch 7/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7558 - accuracy: 0.3470
Epoch 8/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7541 - accuracy: 0.3531
Epoch 9/100
86/86 [==============================] - 0s 4ms/step - loss: 0.7525 - accuracy: 0.3562
Epoch 10/100
86/86 [==============================] - 0s 3ms/step - loss: 0.7507 - accuracy: 0.3628
Epoch 11/

86/86 [==============================] - 0s 3ms/step - loss: 0.6222 - accuracy: 0.7762
Epoch 84/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6205 - accuracy: 0.7806
Epoch 85/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6188 - accuracy: 0.7830
Epoch 86/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6171 - accuracy: 0.7861
Epoch 87/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6155 - accuracy: 0.7901
Epoch 88/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6138 - accuracy: 0.7936
Epoch 89/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6121 - accuracy: 0.7982
Epoch 90/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6104 - accuracy: 0.8007
Epoch 91/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6088 - accuracy: 0.8042
Epoch 92/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6071 - accuracy: 0.8082
Epoch 93/100


86/86 [==============================] - 0s 3ms/step - loss: 0.1580 - accuracy: 0.9681
Epoch 66/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1568 - accuracy: 0.9681
Epoch 67/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1557 - accuracy: 0.9681
Epoch 68/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1546 - accuracy: 0.9681
Epoch 69/100
86/86 [==============================] - 0s 3ms/step - loss: 0.1535 - accuracy: 0.9681
Epoch 70/100
86/86 [==============================] - 0s 3ms/step - loss: 0.1525 - accuracy: 0.9681
Epoch 71/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1514 - accuracy: 0.9681
Epoch 72/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1504 - accuracy: 0.9681
Epoch 73/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1495 - accuracy: 0.9681
Epoch 74/100
86/86 [==============================] - 0s 2ms/step - loss: 0.1485 - accuracy: 0.9681
Epoch 75/100


86/86 [==============================] - 0s 3ms/step - loss: 0.0266 - accuracy: 0.9901
Epoch 48/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0275 - accuracy: 0.9892
Epoch 49/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0260 - accuracy: 0.9901
Epoch 50/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0242 - accuracy: 0.9918
Epoch 51/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0231 - accuracy: 0.9919
Epoch 52/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0232 - accuracy: 0.9925
Epoch 53/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0225 - accuracy: 0.9921
Epoch 54/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0217 - accuracy: 0.9925
Epoch 55/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0202 - accuracy: 0.9929
Epoch 56/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0190 - accuracy: 0.9936
Epoch 57/100


86/86 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 30/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 31/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 32/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 33/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 34/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 35/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 36/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 37/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 38/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 39/100


86/86 [==============================] - 0s 2ms/step - loss: 0.6692 - accuracy: 0.9681
Epoch 10/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6678 - accuracy: 0.9681
Epoch 11/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6665 - accuracy: 0.9681
Epoch 12/100
86/86 [==============================] - 0s 5ms/step - loss: 0.6653 - accuracy: 0.9681
Epoch 13/100
86/86 [==============================] - 0s 5ms/step - loss: 0.6641 - accuracy: 0.9681
Epoch 14/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6629 - accuracy: 0.9681
Epoch 15/100
86/86 [==============================] - 1s 6ms/step - loss: 0.6618 - accuracy: 0.9681
Epoch 16/100
86/86 [==============================] - 1s 6ms/step - loss: 0.6608 - accuracy: 0.9681
Epoch 17/100
86/86 [==============================] - 0s 5ms/step - loss: 0.6597 - accuracy: 0.9681
Epoch 18/100
86/86 [==============================] - 0s 4ms/step - loss: 0.6587 - accuracy: 0.9681
Epoch 19/100


86/86 [==============================] - 0s 3ms/step - loss: 0.6164 - accuracy: 0.9681
Epoch 92/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6159 - accuracy: 0.9681
Epoch 93/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6155 - accuracy: 0.9681
Epoch 94/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6150 - accuracy: 0.9681
Epoch 95/100
86/86 [==============================] - 0s 2ms/step - loss: 0.6146 - accuracy: 0.9681
Epoch 96/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6141 - accuracy: 0.9681
Epoch 97/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6137 - accuracy: 0.9681
Epoch 98/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6132 - accuracy: 0.9681
Epoch 99/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6127 - accuracy: 0.9681
Epoch 100/100
86/86 [==============================] - 0s 3ms/step - loss: 0.6122 - accuracy: 0.9681
Epoch 1/100


86/86 [==============================] - 0s 3ms/step - loss: 0.0416 - accuracy: 0.9850
Epoch 74/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0408 - accuracy: 0.9861
Epoch 75/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0402 - accuracy: 0.9853
Epoch 76/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0407 - accuracy: 0.9853
Epoch 77/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0395 - accuracy: 0.9855
Epoch 78/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0390 - accuracy: 0.9859
Epoch 79/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0384 - accuracy: 0.9864
Epoch 80/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0378 - accuracy: 0.9863
Epoch 81/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0371 - accuracy: 0.9877
Epoch 82/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0367 - accuracy: 0.9868
Epoch 83/100


86/86 [==============================] - 0s 2ms/step - loss: 0.0257 - accuracy: 0.9919
Epoch 56/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0206 - accuracy: 0.9929
Epoch 57/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0286 - accuracy: 0.9918
Epoch 58/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0210 - accuracy: 0.9929
Epoch 59/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0232 - accuracy: 0.9918
Epoch 60/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.9921
Epoch 61/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0214 - accuracy: 0.9923
Epoch 62/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 0.9925
Epoch 63/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0242 - accuracy: 0.9923
Epoch 64/100
86/86 [==============================] - 0s 3ms/step - loss: 0.0275 - accuracy: 0.9925
Epoch 65/100


86/86 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 0.9952
Epoch 38/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 0.9952
Epoch 39/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 0.9954
Epoch 40/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 0.9952
Epoch 41/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9954
Epoch 42/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 0.9952
Epoch 43/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 0.9954
Epoch 44/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9952
Epoch 45/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9954
Epoch 46/100
86/86 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9952
Epoch 47/100


###  Conclusión
**La mejor precisión** la realiza el algoritmo de **Árboles Aleatorios utilizando como criterio el indice Gini** con un **96,99%**.

Teniendo la siguiente **matriz de confusión**:
- ([[1312,    6],[  35,   11]] **Ha acertado correctamente que 1312 no entraran en bancarrota y 11 que si, del test de datos que eran 1364 empresas**.

**De modo que ya podría utilizar este modelo para analizar y predecir otras empresas con estas mismas características.**

In [43]:
 df.sort_values(by='Precision',ascending=False)

,Nombre_Algoritmo,Precision
7,Árboles Aleatorios Gini,96.994135
0,Análisis discriminante lineal,96.774194
3,KNV,96.774194
10,Red Neuronal Artificial Optimizador: Adagrad,96.774194
2,Regresión Logística,96.627566
13,Red Neuronal Artificial Optimizador: Ftrl,96.627566
1,Adaboost,96.407625
6,Árboles Aleatorios Entropia,96.334311
8,XGBOOST,96.260997
14,Red Neuronal Artificial Optimizador: Nadam,95.894428


**Autor: Carlos Mir Martínez**

**Fecha: 20/06/2022**